In [3]:
import os
import pandas as pd
import random
import requests

In [4]:
# Define the paths to your CSV files
good_csv_path = 'goodOutput.csv'
bad_csv_path = 'badOutput.csv'

# Define the output directory structure
output_base_dir = 'output'
output_dirs = ['test/good', 'test/bad', 'train/good', 'train/bad']

# Create the output directories
for output_dir in output_dirs:
    os.makedirs(os.path.join(output_base_dir, output_dir), exist_ok=True)

# Load the CSV files into pandas dataframes
good_df = pd.read_csv(good_csv_path)
bad_df = pd.read_csv(bad_csv_path)

# Define the number of random rows to select for testing
num_test_samples = 50

# Randomly select rows for testing and training from both good and bad dataframes
good_test_samples = random.sample(good_df.index.tolist(), num_test_samples)
bad_test_samples = random.sample(bad_df.index.tolist(), num_test_samples)

# Download and save the test set images
for index in good_test_samples:
    url = good_df.at[index, 'Thumbnail']
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(output_base_dir, 'test/good', f'good_test_{index}.jpg'), 'wb') as f:
            f.write(response.content)

for index in bad_test_samples:
    url = bad_df.at[index, 'Thumbnail']
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(output_base_dir, 'test/bad', f'bad_test_{index}.jpg'), 'wb') as f:
            f.write(response.content)

# Create lists of rows for the training set
good_train_samples = [index for index in good_df.index if index not in good_test_samples]
bad_train_samples = [index for index in bad_df.index if index not in bad_test_samples]

# Download and save the training set images
for index in good_train_samples:
    url = good_df.at[index, 'Thumbnail']
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(output_base_dir, 'train/good', f'good_train_{index}.jpg'), 'wb') as f:
            f.write(response.content)

for index in bad_train_samples:
    url = bad_df.at[index, 'Thumbnail']
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(output_base_dir, 'train/bad', f'bad_train_{index}.jpg'), 'wb') as f:
            f.write(response.content)

In [5]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from tensorflow.keras.preprocessing.image import ImageDataGenerator

We shall use the ImageDataGenerator class to feed in the training and validation data to the model. This class can also be used to generate augmented data.

To know more about ImageDataGenerator class, visit https://keras.io/preprocessing/image/#imagedatagenerator-class

In [6]:
train_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_datagenerator = train_datagenerator.flow_from_directory(
    'output/train',
    target_size=(128,128),
    batch_size=40,
    class_mode='binary')

test_datagenerator = test_datagenerator.flow_from_directory(
    'output/test',
    target_size=(128,128),
    batch_size=10,
    class_mode='binary')

Found 1151 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


Our model will have 3 Convolution2D layers. You can increse or decrease as per your needs.

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D((2,2),2),

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),

    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [8]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 128)       0

We shall use Adam optimizer with a learning rate of 0.001 (hyperparameter). We choose 'binary_crossentropy' loss as our model is a binary calssifier (i.e, we have only 2 classes)

In [9]:
model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.001),
             metrics=['accuracy'])

At the end of each epoch we can check if the model has reached the required accuracy and terminate the training.

In [39]:
DESIRED_ACCURACY = 0.97

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}) :
        if(logs.get('acc') is not None and logs.get('acc') >= DESIRED_ACCURACY) :
            print('\nReached 95% accuracy so cancelling training!')
            self.model.stop_training = True

callbacks = myCallback()

To know more about fit_generator visit https://keras.io/models/model/#fit_generator

In [40]:
model.fit_generator(
    train_datagenerator,
    epochs=100,
    validation_data = test_datagenerator,
    callbacks = [callbacks]
    )

Epoch 1/100


<ipython-input-40-9318e07318eb>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


29/29 [==============================] - 3s 67ms/step - loss: 1.0044e-05 - accuracy: 1.0000 - val_loss: 5.9560 - val_accuracy: 0.8800
Epoch 2/100
29/29 [==============================] - 2s 64ms/step - loss: 9.6991e-06 - accuracy: 1.0000 - val_loss: 5.9717 - val_accuracy: 0.8800
Epoch 3/100
29/29 [==============================] - 2s 63ms/step - loss: 9.4038e-06 - accuracy: 1.0000 - val_loss: 5.9789 - val_accuracy: 0.8800
Epoch 4/100
29/29 [==============================] - 2s 70ms/step - loss: 9.2056e-06 - accuracy: 1.0000 - val_loss: 5.9866 - val_accuracy: 0.8800
Epoch 5/100
29/29 [==============================] - 3s 97ms/step - loss: 8.8461e-06 - accuracy: 1.0000 - val_loss: 5.9995 - val_accuracy: 0.8800
Epoch 6/100
29/29 [==============================] - 3s 88ms/step - loss: 8.5261e-06 - accuracy: 1.0000 - val_loss: 6.0024 - val_accuracy: 0.8800
Epoch 7/100
29/29 [==============================] - 2s 76ms/step - loss: 8.3073e-06 - accuracy: 1.0000 - val_loss: 6.0119 - val_accurac

In [41]:
model.save('mymodel.h5')

In [42]:
model = tf.keras.models.load_model('mymodel.h5')


In [43]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [57]:
path = '4.jpg'
img = image.load_img(path, target_size=(128, 128))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images)
print(classes[0][0])
if classes[0]<0.5:
    print("Given image is bad")
else:
    print("Given image is good")

1/1 [==============================] - 0s 17ms/step
1.0
Given image is good


In [66]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load your pre-trained model
model = tf.keras.models.load_model('mymodel.h5')

# Define the directory path
image_directory = 'shiba'  # Replace with the actual path to your image directory

# Iterate through the image files in the directory
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct the full path to the image
        image_path = os.path.join(image_directory, filename)

        # Load and preprocess the image
        img = image.load_img(image_path, target_size=(128, 128))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # Make predictions
        classes = model.predict(x)
        probability = classes[0][0]

        if probability < 0.5:
            print(f"Image {filename} - Given image is bad")
        else:
            print(f"Image {filename} - Given image is good")


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 76ms/step
Image 5.jpg - Given image is good
1/1 [==============================] - 0s 17ms/step
Image 1.jpg - Given image is good
1/1 [==============================] - 0s 17ms/step
Image 6.jpg - Given image is bad
1/1 [==============================] - 0s 20ms/step
Image 3.jpg - Given image is good
1/1 [==============================] - 0s 19ms/step
Image 4.jpg - Given image is bad
1/1 [==============================] - 0s 18ms/step
Image 2.jpg - Given image is good
